<img src="Screen Shot 2022-03-19 at 16.17.33.png"/>.png"/>

<img src="Screen Shot 2022-03-19 at 16.18.21.png"/>.png"/>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from scipy.spatial import distance_matrix
import sys
import docplex.mp
from docplex.mp.model import Model

In [2]:
NumberCustomers = 10
Capacity = 200
Customers = [i for i in range(1, NumberCustomers + 1)]
CustomersDepot = [0] + Customers + [NumberCustomers + 1]  ##index 0 and 11 are depot 
Vehicles = [i for i in range(1, 4)]
M = 10000  #Big M
ReadyTime=[0,74,151,116,42,65,42,90,167,95,180,0]
DueTime = [240,104,181,146,72,95,72,120,197,125,210,240]
ser=[0,10,10,10,10,10,10,10,10,10,10,0]
PDemand=[0,10,40,10,20,23,16,23,3,25,26,0]
DDemand=[0,10,10,30,19,14,9,13,13,23,3,0]

In [3]:
XCOORD=[40,88,42,72,10,65,27,12,49,57,31,40]
YCOORD=[50,30,5,35,20,55,43,24,42,48,67,50]
A = [(i, j) for i in CustomersDepot for j in CustomersDepot ]
#print(A)
TravelDistance = {(i, j): np.hypot(XCOORD[i]-XCOORD[j], YCOORD[i]-YCOORD[j]) for i, j in A}
Time = {(i, j): np.hypot(XCOORD[i]-XCOORD[j], YCOORD[i]-YCOORD[j]) for i, j in A}
TravelDistance

{(0, 0): 0.0,
 (0, 1): 52.0,
 (0, 2): 45.044422518220834,
 (0, 3): 35.34119409414458,
 (0, 4): 42.42640687119285,
 (0, 5): 25.495097567963924,
 (0, 6): 14.7648230602334,
 (0, 7): 38.2099463490856,
 (0, 8): 12.041594578792296,
 (0, 9): 17.11724276862369,
 (0, 10): 19.235384061671343,
 (0, 11): 0.0,
 (1, 0): 52.0,
 (1, 1): 0.0,
 (1, 2): 52.354560450833695,
 (1, 3): 16.76305461424021,
 (1, 4): 78.63841300535,
 (1, 5): 33.97057550292606,
 (1, 6): 62.369864518050704,
 (1, 7): 76.23647421018367,
 (1, 8): 40.80441152620633,
 (1, 9): 35.84689665786984,
 (1, 10): 67.95586803212802,
 (1, 11): 52.0,
 (2, 0): 45.044422518220834,
 (2, 1): 52.354560450833695,
 (2, 2): 0.0,
 (2, 3): 42.42640687119285,
 (2, 4): 35.34119409414458,
 (2, 5): 55.036351623268054,
 (2, 6): 40.85339643163099,
 (2, 7): 35.510561809129406,
 (2, 8): 37.656340767525464,
 (2, 9): 45.5411901469428,
 (2, 10): 62.96824596572466,
 (2, 11): 45.044422518220834,
 (3, 0): 35.34119409414458,
 (3, 1): 16.76305461424021,
 (3, 2): 42.4264068

In [4]:
time_start = time.time() # Start time
mdl = Model('VRPSPDTW')

In [5]:
X = [(i, j, k) for i in CustomersDepot for j in CustomersDepot for k in Vehicles if i != j] #If vehicle 𝑘 travels from customer 𝑖 to customer 𝑗
Y = [(i, j) for i in CustomersDepot for j in CustomersDepot  ]
Z = [(i, j) for i in CustomersDepot for j in CustomersDepot ]
#Y = [(i, j, k) for i in CustomersDepot for j in CustomersDepot for k in Vehicles if i != j ] #The demand picked up
#Z = [(i, j, k) for i in CustomersDepot for j in CustomersDepot for k in Vehicles if i != j] #The demand to be delivered up to customer
S = [(i, k) for i in CustomersDepot for k in Vehicles] #The beginning service time of customer 𝑖

In [6]:
x = mdl.binary_var_dict(X, name='x')
s = mdl.continuous_var_dict(S, name='s')
y = mdl.continuous_var_dict(Y, name='y')
z = mdl.continuous_var_dict(Z, name='z')

In [7]:
# objective
mdl.sum(TravelDistance[i, j] * x[i, j, k] for i, j, k in X)

docplex.mp.LinearExpr(52x_0_1_1+52x_0_1_2+52x_0_1_3+45.044x_0_2_1+45.044x_0_2_2+45.044x_0_2_3+35.341x_0_3_1+35.341x_0_3_2+35.341x_0_3_3+42.426x_0_4_1+42.426x_0_4_2+42.426x_0_4_3+25.495x_0_5_1+25.495x_0_5_2+25.495x_0_5_3+14.765x_0_6_1+14.765x_0_6_2+14.765x_0_6_3+38.210x_0_7_1+38.210x_0_7_2+38.210x_0_7_3+12.042x_0_8_1+12.042x_0_8_2+12.042x_0_8_3+17.117x_0_9_1+17.117x_0_9_2+17.117x_0_9_3+19.235x_0_10_1+19.235x_0_10_2+19.235x_0_10_3+52x_1_0_1+52x_1_0_2+52x_1_0_3+52.355x_1_2_1+52.355x_1_2_2+52.355x_1_2_3+16.763x_1_3_1+16.763x_1_3_2+16.763x_1_3_3+78.638x_1_4_1+78.638x_1_4_2+78.638x_1_4_3+33.971x_1_5_1+33.971x_1_5_2+33.971x_1_5_3+62.370x_1_6_1+62.370x_1_6_2+62.370x_1_6_3+76.236x_1_7_1+76.236x_1_7_2+76.236x_1_7_3+40.804x_1_8_1+40.804x_1_8_2+40.804x_1_8_3+35.847x_1_9_1+35.847x_1_9_2+35.847x_1_9_3+67.956x_1_10_1+67.956x_1_10_2+67.956x_1_10_3+52x_1_11_1+52x_1_11_2+52x_1_11_3+45.044x_2_0_1+45.044x_2_0_2+45.044x_2_0_3+52.355x_2_1_1+52.355x_2_1_2+52.355x_2_1_3+42.426x_2_3_1+42.426x_2_3_2+42.426x_2_3

In [8]:
# guarantees that each customer is visited exactly once.
mdl.add_constraints(mdl.sum(x[i,j,k] for j in CustomersDepot for k in Vehicles if j!= i )==1 for i in Customers )

[docplex.mp.LinearConstraint[](x_1_0_1+x_1_0_2+x_1_0_3+x_1_2_1+x_1_2_2+x_1_2_3+x_1_3_1+x_1_3_2+x_1_3_3+x_1_4_1+x_1_4_2+x_1_4_3+x_1_5_1+x_1_5_2+x_1_5_3+x_1_6_1+x_1_6_2+x_1_6_3+x_1_7_1+x_1_7_2+x_1_7_3+x_1_8_1+x_1_8_2+x_1_8_3+x_1_9_1+x_1_9_2+x_1_9_3+x_1_10_1+x_1_10_2+x_1_10_3+x_1_11_1+x_1_11_2+x_1_11_3,EQ,1),
 docplex.mp.LinearConstraint[](x_2_0_1+x_2_0_2+x_2_0_3+x_2_1_1+x_2_1_2+x_2_1_3+x_2_3_1+x_2_3_2+x_2_3_3+x_2_4_1+x_2_4_2+x_2_4_3+x_2_5_1+x_2_5_2+x_2_5_3+x_2_6_1+x_2_6_2+x_2_6_3+x_2_7_1+x_2_7_2+x_2_7_3+x_2_8_1+x_2_8_2+x_2_8_3+x_2_9_1+x_2_9_2+x_2_9_3+x_2_10_1+x_2_10_2+x_2_10_3+x_2_11_1+x_2_11_2+x_2_11_3,EQ,1),
 docplex.mp.LinearConstraint[](x_3_0_1+x_3_0_2+x_3_0_3+x_3_1_1+x_3_1_2+x_3_1_3+x_3_2_1+x_3_2_2+x_3_2_3+x_3_4_1+x_3_4_2+x_3_4_3+x_3_5_1+x_3_5_2+x_3_5_3+x_3_6_1+x_3_6_2+x_3_6_3+x_3_7_1+x_3_7_2+x_3_7_3+x_3_8_1+x_3_8_2+x_3_8_3+x_3_9_1+x_3_9_2+x_3_9_3+x_3_10_1+x_3_10_2+x_3_10_3+x_3_11_1+x_3_11_2+x_3_11_3,EQ,1),
 docplex.mp.LinearConstraint[](x_4_0_1+x_4_0_2+x_4_0_3+x_4_1_1+x_4_1_2+x_4_1

In [9]:
# These Constraints state that each vehicle must depart from the depot 0, 
#  leave to another destination after arriving at a customer, and finally, end at the depot 𝑛 + 1.
mdl.add_constraints(
    mdl.sum(x[i,p,k] for i in CustomersDepot if p!=i)
    -mdl.sum(x[p,j,k] for j in CustomersDepot if j!=p)==0 for p in Customers for k in Vehicles)

mdl.add_constraints(mdl.sum(x[0,j,k] for j in Customers if j!=0)==1 for k in Vehicles)
mdl.add_constraints(mdl.sum(x[i,NumberCustomers+1,k] for i in CustomersDepot if i!= NumberCustomers + 1)
                    == 1 for k in Vehicles ) 

[docplex.mp.LinearConstraint[](x_0_11_1+x_1_11_1+x_2_11_1+x_3_11_1+x_4_11_1+x_5_11_1+x_6_11_1+x_7_11_1+x_8_11_1+x_9_11_1+x_10_11_1,EQ,1),
 docplex.mp.LinearConstraint[](x_0_11_2+x_1_11_2+x_2_11_2+x_3_11_2+x_4_11_2+x_5_11_2+x_6_11_2+x_7_11_2+x_8_11_2+x_9_11_2+x_10_11_2,EQ,1),
 docplex.mp.LinearConstraint[](x_0_11_3+x_1_11_3+x_2_11_3+x_3_11_3+x_4_11_3+x_5_11_3+x_6_11_3+x_7_11_3+x_8_11_3+x_9_11_3+x_10_11_3,EQ,1)]

In [10]:
#                  indicates the relationship between departure time at a customer 𝑗 with 
#              departure time, service time and travelling time from its predecessor 𝑖 in a tour.
mdl.add_constraints(s[i,k]+ser[i]+Time[i,j]- M * (1-x[i,j,k]) - s[j,k]
                    <=1 for i,j,k in X if i!=j)
mdl.add_constraints(s[i, k] >= ReadyTime[i] for i, k in S if i != 0)

mdl.add_constraints(s[i, k] <= DueTime[i] for i, k in S if i != 0)


[docplex.mp.LinearConstraint[](s_1_1,LE,104),
 docplex.mp.LinearConstraint[](s_1_2,LE,104),
 docplex.mp.LinearConstraint[](s_1_3,LE,104),
 docplex.mp.LinearConstraint[](s_2_1,LE,181),
 docplex.mp.LinearConstraint[](s_2_2,LE,181),
 docplex.mp.LinearConstraint[](s_2_3,LE,181),
 docplex.mp.LinearConstraint[](s_3_1,LE,146),
 docplex.mp.LinearConstraint[](s_3_2,LE,146),
 docplex.mp.LinearConstraint[](s_3_3,LE,146),
 docplex.mp.LinearConstraint[](s_4_1,LE,72),
 docplex.mp.LinearConstraint[](s_4_2,LE,72),
 docplex.mp.LinearConstraint[](s_4_3,LE,72),
 docplex.mp.LinearConstraint[](s_5_1,LE,95),
 docplex.mp.LinearConstraint[](s_5_2,LE,95),
 docplex.mp.LinearConstraint[](s_5_3,LE,95),
 docplex.mp.LinearConstraint[](s_6_1,LE,72),
 docplex.mp.LinearConstraint[](s_6_2,LE,72),
 docplex.mp.LinearConstraint[](s_6_3,LE,72),
 docplex.mp.LinearConstraint[](s_7_1,LE,120),
 docplex.mp.LinearConstraint[](s_7_2,LE,120),
 docplex.mp.LinearConstraint[](s_7_3,LE,120),
 docplex.mp.LinearConstraint[](s_8_1,LE,197

In [11]:
mdl.add_constraints(mdl.sum(y[j,i] for i in CustomersDepot if i!=j)
                    - mdl.sum(y[i,j] for i in CustomersDepot if i!=j) 
                    == PDemand[j] for j in Customers)
mdl.add_constraints(mdl.sum(z[i,j] for i in CustomersDepot if i!=j)
                    - mdl.sum(z[j,i] for i in CustomersDepot if i!=j) 
                    == DDemand[j] for j in Customers)
mdl.add_constraints(y[i,j]+z[i,j]
                    <=Capacity*mdl.sum(x[i,j,k]for k in Vehicles if i!=j) for i in CustomersDepot for j in CustomersDepot)

[docplex.mp.LinearConstraint[](y_0_0+z_0_0,LE,0),
 docplex.mp.LinearConstraint[](y_0_1+z_0_1,LE,200x_0_1_1+200x_0_1_2+200x_0_1_3),
 docplex.mp.LinearConstraint[](y_0_2+z_0_2,LE,200x_0_2_1+200x_0_2_2+200x_0_2_3),
 docplex.mp.LinearConstraint[](y_0_3+z_0_3,LE,200x_0_3_1+200x_0_3_2+200x_0_3_3),
 docplex.mp.LinearConstraint[](y_0_4+z_0_4,LE,200x_0_4_1+200x_0_4_2+200x_0_4_3),
 docplex.mp.LinearConstraint[](y_0_5+z_0_5,LE,200x_0_5_1+200x_0_5_2+200x_0_5_3),
 docplex.mp.LinearConstraint[](y_0_6+z_0_6,LE,200x_0_6_1+200x_0_6_2+200x_0_6_3),
 docplex.mp.LinearConstraint[](y_0_7+z_0_7,LE,200x_0_7_1+200x_0_7_2+200x_0_7_3),
 docplex.mp.LinearConstraint[](y_0_8+z_0_8,LE,200x_0_8_1+200x_0_8_2+200x_0_8_3),
 docplex.mp.LinearConstraint[](y_0_9+z_0_9,LE,200x_0_9_1+200x_0_9_2+200x_0_9_3),
 docplex.mp.LinearConstraint[](y_0_10+z_0_10,LE,200x_0_10_1+200x_0_10_2+200x_0_10_3),
 docplex.mp.LinearConstraint[](y_0_11+z_0_11,LE,200x_0_11_1+200x_0_11_2+200x_0_11_3),
 docplex.mp.LinearConstraint[](y_1_0+z_1_0,LE,200

In [12]:
    # Objective Function
obj_function = mdl.sum(TravelDistance[i, j] * x[i, j, k] for i, j, k in X)

    # Set time limit
mdl.parameters.timelimit.set(100)

    # Solve
mdl.minimize(obj_function)

time_solve = time.time()

solution = mdl.solve(log_output = True)

time_end = time.time()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               100
Tried aggregator 1 time.
MIP Presolve eliminated 346 rows and 311 columns.
MIP Presolve modified 354 coefficients.
Reduced MIP has 326 rows, 409 columns, and 1693 nonzeros.
Reduced MIP has 225 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (1.11 ticks)
Found incumbent of value 402.645974 after 0.17 sec. (2.92 ticks)
Probing fixed 48 vars, tightened 242 bounds.
Probing time = 0.03 sec. (3.01 ticks)
Cover probing fixed 0 vars, tightened 25 bounds.
Tried aggregator 2 times.
MIP Presolve eliminated 83 rows and 82 columns.
MIP Presolve modified 162 coefficients.
Aggregator did 4 substitutions.
Reduced MIP has 239 rows, 323 columns, and 1297 nonzeros.
Reduced MIP has 177 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.37 ticks)
Probing fixed 12 vars, tightened 12 bounds.
Probing ti

In [13]:
if solution != None:
    obj = round(obj_function.solution_value, 2) # to access the obj
running_time = round(time_end - time_solve, 2)
elapsed_time = round(time_end - time_start, 2)
print('----------------------------------------------------------') 
print('Best_Solution:',obj,' NumberVehicles: ',len(Vehicles),' RunningTime: ',running_time,' ElapsedTime: ',elapsed_time)
print('----------------------------------------------------------')      
active_arcs = [a for a in X if x[a].solution_value > 0.9]
print('Routes: ',active_arcs)
print('----------------------------------------------------------')  
    

----------------------------------------------------------
Best_Solution: 348.98  NumberVehicles:  3  RunningTime:  0.52  ElapsedTime:  1.6
----------------------------------------------------------
Routes:  [(0, 1, 3), (0, 4, 1), (0, 6, 2), (1, 3, 3), (2, 11, 1), (3, 8, 3), (4, 7, 1), (5, 9, 2), (6, 5, 2), (7, 2, 1), (8, 11, 3), (9, 10, 2), (10, 11, 2)]
----------------------------------------------------------


<img src="Screen Shot 2022-03-19 at 16.17.20.png"/>.png"/>